In [1]:
from dbscan_parallel import *

24/05/14 18:20:59 WARN Utils: Your hostname, Yzeys-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 172.30.192.255 instead (on interface en0)
24/05/14 18:20:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/05/14 18:21:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from sklearn.datasets import make_blobs

# Generate random dataset with 3 dimensions
n_samples = 100  # Number of samples
n_features = 3  # Number of dimensions
centers = 3  # Number of clusters
random_state = 42  # Random state for reproducibility

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers, random_state=random_state)

In [3]:
X.shape

(100, 3)

In [4]:
np.max(X, axis=0)

array([ 4.5123324 , 10.41824487,  8.01217509])

In [5]:
np.min(X, axis=0)

array([-10.83328933,  -8.24949825,  -8.02990217])

In [6]:
min_pts = 18
eps = 3
dataset = X
b_dataset = sc.broadcast(dataset)
b_eps = sc.broadcast(eps)
b_min_pts = sc.broadcast(min_pts)

rdd = spatial_partition(X, (3,3,3), eps)


spatial_partition time cost: 456.26091957092285ms


In [39]:
@staticmethod
def local_dbscan(partioned_rdd, method='matrix', metric='euclidian'):

    dataset = np.array([b_dataset.value[idp] for idp in partioned_rdd])
    if method == 'matrix':
        dbscan_obj = MatrixDBSCAN(dataset, b_eps.value, b_min_pts.value, metric) 
    else:
        dbscan_obj = NaiveDBSCAN(dataset, b_eps.value, b_min_pts.value, metric) 
    dbscan_obj.predict()
    is_core_list = dbscan_obj._find_core_pts()
    
    return list(zip(partioned_rdd, is_core_list, dbscan_obj.tags))


local_tags = rdd.mapValues(lambda x: local_dbscan(x, method='matrix', metric='euclidian')).collect()

predict time cost: 8.816719055175781ms                              (0 + 8) / 8]
predict time cost: 9.157180786132812ms
predict time cost: 9.426355361938477ms
predict time cost: 8.928060531616211ms
predict time cost: 0.0362396240234375ms
predict time cost: 0.030994415283203125ms
predict time cost: 0.2739429473876953ms
predict time cost: 0.27489662170410156ms
predict time cost: 0.012159347534179688ms
predict time cost: 12.25590705871582ms
predict time cost: 4.436969757080078ms
predict time cost: 17.14468002319336ms
predict time cost: 0.308990478515625ms
predict time cost: 0.017881393432617188ms
predict time cost: 0.035762786865234375ms
predict time cost: 19.74034309387207ms
predict time cost: 20.066022872924805ms
predict time cost: 3.0608177185058594ms
predict time cost: 3.049135208129883ms
predict time cost: 8.835077285766602ms


In [20]:
local_tags

[(24,
  [((0, 0), 2),
   ((1, 1), 1),
   ((2, 1), 1),
   ((7, 0), 1),
   ((15, 0), 2),
   ((19, 1), 1),
   ((20, 1), 1),
   ((22, 1), 2),
   ((24, 1), 1),
   ((26, 1), 1),
   ((27, 0), 2),
   ((29, 1), 1),
   ((33, 1), 2),
   ((34, 0), 2),
   ((35, 0), 2),
   ((36, 1), 1),
   ((38, 1), 1),
   ((39, 1), 1),
   ((40, 0), 2),
   ((42, 1), 1),
   ((43, 1), 1),
   ((45, 0), 1),
   ((47, 0), 2),
   ((49, 1), 1),
   ((52, 1), 1),
   ((53, 0), 2),
   ((55, 0), 2),
   ((58, 0), 2),
   ((59, 1), 1),
   ((61, 0), 2),
   ((63, 1), 2),
   ((64, 1), 2),
   ((65, 1), 1),
   ((68, 1), 1),
   ((71, 1), 1),
   ((72, 0), 2),
   ((74, 0), 1),
   ((75, 0), 2),
   ((76, 1), 1),
   ((77, 1), 1),
   ((80, 1), 1),
   ((82, 1), 1),
   ((83, 1), 2),
   ((84, 1), 1),
   ((85, 1), 1),
   ((92, 1), 1),
   ((93, 1), 1),
   ((94, 0), 1),
   ((96, 0), 2),
   ((97, 0), 2),
   ((98, 0), 1),
   ((99, 1), 1)]),
 (16,
  [((2, 0), -2),
   ((3, 1), 1),
   ((8, 1), 1),
   ((13, 1), 1),
   ((15, 1), 1),
   ((17, 1), 1),
   ((1

In [41]:
@timeit
def merge(local_tags, dataset):
    global_tags = [UNKNOWN] * len(dataset)
    is_tagged = [0] * len(dataset)
    last_max_label = 0
    for local in local_tags:
        np_local = np.array(local[-1])
        np_local[:, -1] += last_max_label

        last_max_label = np.max(np_local[:, -1])
        
        # check and merge overlapped points
        tagged_indices = np.nonzero(is_tagged)[0]
        for tmp_i in range(len(np_local)):
            # should do tag check
            p_id, is_core, label = np_local[tmp_i]
            if p_id in tagged_indices and is_core==1:
                np_local[-1][np_local[-1]==label] = global_tags[p_id]
        
        # update global tags
        for p_id, is_core, label in np_local:
            if is_tagged[p_id]==1:
                continue
            global_tags[p_id] = label
            is_tagged[p_id] = 1
    return global_tags

result_tags = merge(local_tags, dataset)


merge time cost: 5.343914031982422ms


In [44]:
len(result_tags)

100

In [46]:
rdd.collect()

[[24,
  [0,
   1,
   2,
   7,
   15,
   19,
   20,
   22,
   24,
   26,
   27,
   29,
   33,
   34,
   35,
   36,
   38,
   39,
   40,
   42,
   43,
   45,
   47,
   49,
   52,
   53,
   55,
   58,
   59,
   61,
   63,
   64,
   65,
   68,
   71,
   72,
   74,
   75,
   76,
   77,
   80,
   82,
   83,
   84,
   85,
   92,
   93,
   94,
   96,
   97,
   98,
   99]],
 [16,
  [2,
   3,
   8,
   13,
   15,
   17,
   19,
   21,
   22,
   25,
   27,
   33,
   34,
   35,
   36,
   38,
   40,
   45,
   47,
   49,
   52,
   53,
   55,
   56,
   63,
   64,
   68,
   71,
   72,
   75,
   76,
   78,
   80,
   81,
   82,
   83,
   84,
   85,
   87,
   89,
   91,
   93,
   94,
   96,
   97]],
 [25,
  [0,
   2,
   3,
   8,
   12,
   13,
   15,
   17,
   19,
   21,
   22,
   25,
   27,
   33,
   34,
   35,
   36,
   38,
   40,
   41,
   45,
   47,
   49,
   51,
   52,
   53,
   55,
   56,
   58,
   61,
   63,
   64,
   68,
   71,
   72,
   75,
   76,
   78,
   80,
   81,
   82,
   83,
   84,
   85,
  